In [ ]:
import os
import sys
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import cv2
import gc
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

sys.path.append("../input/efficientnet")

from  efficientnet_pytorch import EfficientNet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
device

In [ ]:
def seed_everything():
    np.random.seed(10)
    random.seed(10)
    torch.manual_seed(10)

seed_everything()

In [ ]:
train_folder="../input/pawpularity-resize-256/resized"
train_df=pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
print(train_df.shape)

In [ ]:
def get_bin_number(x):
    if x<=25:
        return "<=25"
    elif x>25 and x<=35:
        return "25-35"
    elif x>35 and x<=40:
        return "35-45"
    elif x>40 and x<=60:
        return "45-60"
    elif x>60 and x<=75:
        return "60-75"
    else:
        return ">75"

In [ ]:
train_df['bin_num'] = train_df.Pawpularity.apply(lambda x: min(9, x//10))
train_df.head()

In [ ]:
train_df.bin_num.unique().shape

In [ ]:
sns.countplot(data=train_df, x="bin_num",
              order=np.arange(10))

In [ ]:
train_df['score'] = train_df.Pawpularity/100
sns.histplot(train_df.score)

In [ ]:
def kfold(train_df, k=5):
    image_ids=train_df.Id.values
    bins=train_df.bin_num.unique()
    train_df=train_df.sample(frac=1.0, random_state=22)
    fold_map={}
    
    for bin_num in bins:
        image_ids=train_df[train_df.bin_num == bin_num].Id.values
        num_images=len(image_ids)
        slice_length=num_images//k
        for i in range(0, num_images, slice_length):
            fold_num=min(i//slice_length, k-1)
            for j in range(i, i+slice_length):
                if j >= num_images:
                    break
                fold_map[ image_ids[j] ] = fold_num
    
    df=train_df.copy()
    df['fold'] = df['Id'].apply(lambda x: fold_map[x])
    return df

# Transformations

In [ ]:
train_transform=A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(p=0.7, limit=(-20, 20), border_mode=2),
    A.RGBShift(p=1.0, r_shift_limit=(-15, 15),
               g_shift_limit=(-15, 15),
               b_shift_limit=(-15, 15)
              ),
    A.RandomBrightnessContrast(p=1.0),
    A.CoarseDropout(p=1.0,min_holes=5, max_holes=10,
                    min_width=8, max_width=12,
                    min_height=8, max_height=12),
    
    A.Normalize(p=1.0)
])
val_transform = A.Compose([A.Normalize(p=1.0)])

# Read Images

In [ ]:
def read_image(image_name, phase):
    filepath=os.path.join(train_folder, "{}.jpg".format(image_name))
    img=cv2.imread(filepath)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if phase!='eval':
        img=train_transform(image = img)['image']
    else:
        img=val_transform(image = img)['image']
    return img

# Model

In [ ]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.efficient_net=EfficientNet.from_pretrained('efficientnet-b2', include_top=True)
        self.avg_pooling=nn.AdaptiveAvgPool2d(1)
        
        self.fc=nn.Sequential(
            nn.BatchNorm1d(1408),
            nn.Linear(1408, 1024),
            nn.SiLU(),
            nn.Dropout(0.2),
            
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 1024),
            nn.SiLU(),
            nn.Dropout(0.2),
            
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 1)
        )
    def forward(self, x):
        batch_size=x.size(0)
        x=self.efficient_net.extract_features(x)
        xpool=self.avg_pooling(x).view(batch_size, -1)
        x=self.fc(xpool)
        x=torch.sigmoid(x)
        return (xpool, x)

# Dataset and dataloaders

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, image_ids, scores, phase):
        self.image_ids=image_ids
        self.scores=scores
        self.phase=phase
    def __getitem__(self, idx):
        image_name = self.image_ids[idx]
        score=self.scores[idx]
        if self.phase == 'train':
            score=min(0.95, score)
            score=max(score, 0.05)
            score=score + np.random.uniform(-0.03, 0.03)
            
        img=read_image(image_name, self.phase)
        
        X=torch.tensor(img, dtype=torch.float32).transpose(0, 2)
        y=torch.tensor(score, dtype=torch.float32)
        return (X, y)
        
    def __len__(self):
        return len(self.image_ids)

In [ ]:
def get_dataloaders(fold_num, df):
    BATCH_SIZE=32
    
    train_image_ids=df[df.fold!=fold_num].Id.values
    train_scores=df[df.fold!=fold_num].score.values
    
    val_image_ids=df[df.fold==fold_num].Id.values
    val_scores=df[df.fold==fold_num].score.values
    
    train_dataset=Dataset(train_image_ids, train_scores, phase='train')
    val_dataset=Dataset(val_image_ids, val_scores, phase='eval')
    
    train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
    val_dataloader=torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, pin_memory=True)
    
    return (train_dataloader, val_dataloader)

# training

In [ ]:
def rmse(y, yhat):
    yerr=torch.abs(y-yhat)
    rmse_loss=torch.sqrt( torch.mean( yerr**2 ) )
    return rmse_loss

In [ ]:
def evaluate(val_dataloader, model):
    model.eval()
    eval_loss = 0
    for it, (X, y) in enumerate(val_dataloader):
        X=X.to(device)
        y=y.to(device)
        
        with torch.no_grad():
            (_, yhat)=model(X)
            yhat=yhat.view(-1)
            rmse_loss=rmse(y, yhat)
            eval_loss+=rmse_loss
    eval_loss/=len(val_dataloader)
    eval_loss = 100 * eval_loss
    return eval_loss

In [ ]:
def train_epoch(train_dataloader, optimizer, schedular, model):
    model.train()
    below_thresh_count=0
    epoch_loss=0.0
    epoch_rmse_loss=0.0
    for it, (X, y) in enumerate(train_dataloader):
        X=X.to(device)
        y=y.to(device)
        
        bsize=X.shape[0]
        (_, yhat)=model(X)
        yhat=yhat.view(-1)
        
        rmse_loss=rmse(y, yhat)
        ydiff = torch.abs(y-yhat)
        ydiff_thresh = ydiff#[ydiff>0.0001]
        
        optimizer.zero_grad(set_to_none=True)
        loss = 10 * ydiff_thresh.mean()
        loss.backward()
        optimizer.step()
        schedular.step()
        below_thresh_count+= ( bsize - ydiff_thresh.shape[0] )
        epoch_loss += loss.item()
        epoch_rmse_loss+= 100*rmse_loss.item()
        
        del X
        del y
        del yhat
    
    epoch_loss/=len(train_dataloader)
    epoch_rmse_loss/=len(train_dataloader)
    return (epoch_loss, epoch_rmse_loss, below_thresh_count)

In [ ]:
def train_model(fold_num, train_dataloader,val_dataloader, model):
    epochs=20
    patience=7
    max_patience=7
    max_lr=2e-5
    
    best_loss=None
    optimizer=torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-6)
    schedular=torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                  max_lr=max_lr,
                                                  epochs=epochs,
                                                  steps_per_epoch=len(train_dataloader))
    
    for e in range(epochs):
        if patience == 0:
            print("Early Stopped Trainig at epoch:", e+1)
            break
            
        (epoch_loss, epoch_rmse_loss, below_thresh_count) = train_epoch(train_dataloader, optimizer, schedular, model)
        (eval_rmse) = evaluate(val_dataloader, model)
        gc.collect()
        print("Epoch:{}|Train Loss:{:.4f} | train rmseloss:{:.4f} | Below Thresh:{}".format(e+1, epoch_loss, 
                                                                                            epoch_rmse_loss,
                                                                                            below_thresh_count))
        print("Eval RMSE:{:4f}".format(eval_rmse))
        
        if (best_loss is not None) and (eval_rmse > best_loss):
            patience-=1
            print("Patience:",patience)
            
        if (best_loss is None) or ( eval_rmse < best_loss ):
            best_loss = eval_rmse
            patience=max_patience
            torch.save(model, "model{}.pth".format(fold_num+1))

In [ ]:
def lr_range_test():
    model=Baseline()
    model=model.to(device)
    
    min_lr=7e-6
    max_lr=1e-3
    
    optimizer=torch.optim.AdamW(model.parameters(), lr=min_lr, weight_decay=1e-5)
    train_fold_df=kfold(train_df)
    (train_dataloader, _) = get_dataloaders(0, train_fold_df)
    
    lrs=[]
    losses=[]
    model.train()
    while min_lr < max_lr:
        for it, (X,y) in enumerate(train_dataloader):
            X=X.to(device)
            y=y.to(device)
            
            min_lr=min_lr * 1.02
            if min_lr > max_lr:
                break
            
            (_, yhat)=model(X)
            yhat=yhat.view(-1)
            ydiff = torch.abs(y-yhat)
            ydiff_thresh = ydiff

            optimizer.zero_grad(set_to_none=True)
            loss = 10 * ydiff_thresh.mean()
            loss.backward()
            optimizer.step()
            if it%10==0:
                print(min_lr, loss.item())
            optimizer.param_groups[0]['lr'] = min_lr
            
            lrs.append(min_lr)
            losses.append(loss.item())
    return (lrs, losses)

In [ ]:
#(lrs, losses)=lr_range_test()
#plt.plot(lrs, losses)

In [ ]:
#lmt=50
#plt.plot(lrs[:lmt], losses[:lmt])

In [ ]:
kfolds=5
train_fold_df=kfold(train_df)
for fold_num in train_fold_df.fold.unique():
    model=Baseline()
    model=model.to(device)
    (train_dataloader, val_dataloader) = get_dataloaders(fold_num, train_fold_df)
    train_model(fold_num, train_dataloader, val_dataloader, model)
    break